## Part 1 Download Datset and Understand the Format

Import Libraries

In [10]:
import gzip
import numpy as np
from sklearn.preprocessing import scale
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import pairwise_distances


Read Dataset files

In [11]:
trainDataset = gzip.open('kddcup.data_10_percent.gz', 'r').readlines()
testDataset = gzip.open('corrected.gz', 'r').readlines()

Maps from categorical data to numerical


In [12]:
attrNames = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label']
protocol_type = {'tcp': 0, 'udp': 1, 'icmp': 2}
service = {'aol': 0, 'auth': 1, 'bgp': 2, 'courier': 3, 'csnet_ns': 4, 'ctf': 5, 'daytime': 6, 'discard': 7, 'domain': 8, 'domain_u': 9, 'echo': 10, 'eco_i': 11, 'ecr_i': 12, 'efs': 13, 'exec': 14, 'finger': 15, 'ftp': 16, 'ftp_data': 17, 'gopher': 18, 'harvest': 19, 'hostnames': 20, 'http': 21, 'http_2784': 22, 'http_443': 23, 'http_8001': 24, 'imap4': 25, 'IRC': 26, 'iso_tsap': 27, 'klogin': 28, 'kshell': 29, 'ldap': 30, 'link': 31, 'login': 32, 'mtp': 33, 'name': 34, 'netbios_dgm': 35, 'netbios_ns': 36, 'netbios_ssn': 37, 'netstat': 38, 'nnsp': 39, 'nntp': 40, 'ntp_u': 41, 'other': 42, 'pm_dump': 43, 'pop_2': 44, 'pop_3': 45, 'printer': 46, 'private': 47, 'red_i': 48, 'remote_job': 49, 'rje': 50, 'shell': 51, 'smtp': 52, 'sql_net': 53, 'ssh': 54, 'sunrpc': 55, 'supdup': 56, 'systat': 57, 'telnet': 58, 'tftp_u': 59, 'tim_i': 60, 'time': 61, 'urh_i': 62, 'urp_i': 63, 'uucp': 64, 'uucp_path': 65, 'vmnet': 66, 'whois': 67, 'X11': 68, 'Z39_50': 69, 'icmp': 70}
flag = {'OTH': 0, 'REJ': 1, 'RSTO': 2, 'RSTOS0': 3, 'RSTR': 4, 'S0': 5, 'S1': 6, 'S2': 7, 'S3': 8, 'SF': 9, 'SH': 10}
labels = {'normal': 0, 'back': 1, 'buffer_overflow': 2, 'ftp_write': 3, 'guess_passwd': 4, 'imap': 5, 'ipsweep': 6, 'land': 7, 'loadmodule': 8, 'multihop': 9, 'neptune': 10, 'nmap': 11, 'perl': 12, 'phf': 13, 'pod': 14, 'portsweep': 15, 'rootkit': 16, 'satan': 17, 'smurf': 18, 'spy': 19, 'teardrop': 20, 'warezclient': 21, 'warezmaster': 22, 'snmpgetattack': 23, 'snmpguess': 24, 'httptunnel': 25, 'sendmail': 26, 'named': 27, 'xlock': 28, 'xsnoop': 29, 'worm': 30, 'xterm': 31, 'ps': 32, 'sqlattack': 33, 'udpstorm': 34, 'mailbomb': 35, 'saint': 36, 'apache2': 37, 'mscan': 38, 'processtable': 39,'icmp': 40}

Change the categorical features to numerical

In [13]:
for i in range(len(trainDataset)):
    
    trainDataset[i] = trainDataset[i].decode('utf-8') # convert from bytes to string
    if trainDataset[i].endswith('.\n'): 
        trainDataset[i] = trainDataset[i].replace('.\n', '') # remove '.\n'
    trainDataset[i] = trainDataset[i].strip().split(',') # split by comma
    
    for j in range(len(trainDataset[i])):
        try:
            trainDataset[i][j] = int(trainDataset[i][j]) # convert to int
        except ValueError:
            try:
                trainDataset[i][j] = float(trainDataset[i][j]) # convert to float
            except ValueError: # convert categorical data to numerical data
                if j == 1: 
                    trainDataset[i][j] = protocol_type[trainDataset[i][j]]
                elif j == 2:
                    trainDataset[i][j] = service[trainDataset[i][j]]
                elif j == 3:
                    trainDataset[i][j] = flag[trainDataset[i][j]]
                elif j == 41:
                    trainDataset[i][j] = labels[trainDataset[i][j]]

In [14]:
# convert from bytes to string and change categorical data to numerical data
for i in range(len(testDataset)):
    
    testDataset[i] = testDataset[i].decode('utf-8') # convert from bytes to string
    if testDataset[i].endswith('.\n'):
        testDataset[i] = testDataset[i].replace('.\n', '') # remove '.\n'
    testDataset[i] = testDataset[i].strip().split(',') # split by comma
    
    for j in range(len(testDataset[i])):
        try:
            testDataset[i][j] = int(testDataset[i][j]) # convert to int
        except ValueError:
            try:
                testDataset[i][j] = float(testDataset[i][j]) # convert to float
            except ValueError: # convert categorical data to numerical data
                if j == 1:
                    testDataset[i][j] = protocol_type[testDataset[i][j]]
                elif j == 2:
                    testDataset[i][j] = service[testDataset[i][j]]
                elif j == 3:
                    testDataset[i][j] = flag[testDataset[i][j]]
                elif j == 41:
                    testDataset[i][j] = labels[testDataset[i][j]]

In [15]:
# remove duplicate rows
trainDataset = list(set(tuple(row) for row in trainDataset))
testDataset = list(set(tuple(row) for row in testDataset))

trainDataset = list(list(row) for row in trainDataset)
testDataset = list(list(row) for row in testDataset)

In [16]:
# Split the dataset into features and labels
testLabels = []
for i in range(len(testDataset)):
    testLabels.append(testDataset[i][41])
    testDataset[i].pop(41)
trainLabels = []
for i in range(len(trainDataset)):
    trainLabels.append(trainDataset[i][41])
    trainDataset[i].pop(41)

In [17]:
# take 0.01 from trainDataset for testing purposes
train10 = trainDataset[:int(len(trainDataset)*0.01)]
trainLabels10 = trainLabels[:int(len(trainLabels)*0.01)]
test10 = testDataset[:int(len(testDataset)*0.01)]
testLabels10 = testLabels[:int(len(testLabels)*0.01)]

In [19]:
scaled_train10 = scale(train10, axis=0, with_mean=True, with_std=True, copy=True)
scaled_test10 = scale(test10, axis=0, with_mean=True, with_std=True, copy=True)
scaled_train = scale(trainDataset, axis=0, with_mean=True, with_std=True, copy=True)
scaled_test = scale(testDataset, axis=0, with_mean=True, with_std=True, copy=True)

In [20]:
class k_medoids:
    def __init__(self, k = 2, max_iter = 300, has_converged = False):
        self.k = k
        self.max_iter = max_iter
        self.has_converged = has_converged
        self.medoids_cost = []
        
    def initMedoids(self, X):
        
        self.medoids = []
        
        indexes = np.random.randint(0, len(X)-1,self.k)
        self.medoids = X[indexes]
        
        for i in range(0,self.k):
            self.medoids_cost.append(0)
        
    def isConverged(self, new_medoids):
        
        return set([tuple(x) for x in self.medoids]) == set([tuple(x) for x in new_medoids])
        
    def updateMedoids(self, X, labels):
        self.has_converged = True
        
        clusters = []
        for i in range(0,self.k):
            cluster = []
            for j in range(len(X)):
                if (labels[j] == i):
                    cluster.append(X[j])
            clusters.append(cluster)
        
        new_medoids = []
        for i in range(0, self.k):
            new_medoid = self.medoids[i]
            old_medoids_cost = self.medoids_cost[i]
            for j in range(len(clusters[i])):
                
                cur_medoids_cost = 0
                for dpoint_index in range(len(clusters[i])):
                    cur_medoids_cost += pairwise_distances(np.array(clusters[i][j]).reshape(1,-1), np.array(clusters[i][dpoint_index]).reshape(1,-1))
                
                if cur_medoids_cost < old_medoids_cost:
                    new_medoid = clusters[i][j]
                    old_medoids_cost = cur_medoids_cost
            
            new_medoids.append(new_medoid)
        
        if not self.isConverged(new_medoids):
            self.medoids = new_medoids
            self.has_converged = False
    
    def fit(self, X):
        self.initMedoids(X)
        
        for i in tqdm(range(self.max_iter)):
            cur_labels = []
            for medoid in range(0,self.k):
                self.medoids_cost[medoid] = 0
                for k in range(len(X)):
                    d_list = []                    
                    for j in range(0,self.k):
                        d_list.append(pairwise_distances(np.array(self.medoids[j]).reshape(1,-1), np.array(X[k]).reshape(1,-1)))
                    cur_labels.append(d_list.index(min(d_list)))
                    
                    self.medoids_cost[medoid] += min(d_list)
                                
            self.updateMedoids(X, cur_labels)
            
            if self.has_converged:
                break

        return np.array(self.medoids)

        
    def predict(self,data):
        pred = []
        for i in range(len(data)):
            d_list = []
            for j in range(len(self.medoids)):
                d_list.append(pairwise_distances(np.array(self.medoids[j]).reshape(1,-1),np.array(data[i]).reshape(1,-1)))
                
            pred.append(d_list.index(min(d_list)))
            
        return np.array(pred)

In [21]:
d = k_medoids(k=15, max_iter = 20)

In [22]:
meds = d.fit(np.array(scaled_train[0:100]))

 20%|██        | 4/20 [00:20<01:22,  5.17s/it]


In [23]:
clusterLabels = {}
trainClusters = d.predict(np.array(scaled_train[0:100]))
medLabels = []
for i in tqdm(range(len(trainClusters))):
    if trainClusters[i] not in clusterLabels.keys():
        clusterLabels[trainClusters[i]] = [trainLabels10[i]]
    else:
        clusterLabels[trainClusters[i]].append(trainLabels10[i])

for key in clusterLabels:
    medLabels.append(max(set(clusterLabels[key]), key = clusterLabels[key].count))

100%|██████████| 100/100 [00:00<00:00, 197564.96it/s]


In [24]:
result = d.predict(np.array(scaled_test))

In [33]:
def calcAccuracy(predict, trueLabels, medLabels):
    acc = 0
    for i in range(len(predict)):
        if trueLabels[i] == 0 and medLabels[predict[i]] == 0:
            acc += 1
        elif trueLabels[i] != 0 and medLabels[predict[i]] != 0:
            acc += 1
    return acc/len(predict)

In [39]:
# calculate precision
def calcPercision(predict, trueLabels,medLabels):
    TP = 0
    FP = 0
    for i in range(len(predict)):
        if trueLabels[i] == 0 and medLabels[predict[i]] == 0:
            TP += 1
        elif trueLabels[i] == 0 and medLabels[predict[i]] != 0:
            FP += 1
    return TP/(TP+FP)

In [44]:
# calculate recall
def calcRecall(predict, trueLabels,medLabels):
    TP = 0
    FN = 0
    for i in range(len(predict)):
        if trueLabels[i] == 0 and medLabels[predict[i]] == 0:
            TP += 1
        elif trueLabels[i] != 0 and medLabels[predict[i]] == 0:
            FN += 1
    return TP/(TP+FN)

In [36]:
# calculate F1 score
def calcF1(precision, recall):
    return 2*precision*recall/(precision+recall)

In [43]:
# calulate conditional entropy
def calcEntropy(predict, trueLabels,medLabels):
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    for i in range(len(predict)):
        if trueLabels[i] == 0 and medLabels[predict[i]] == 0:
            TP += 1
        elif trueLabels[i] == 0 and medLabels[predict[i]] != 0:
            FP += 1
        elif trueLabels[i] != 0 and medLabels[predict[i]] == 0:
            FN += 1
        elif trueLabels[i] != 0 and medLabels[predict[i]] != 0:
            TN += 1
    return -TP/len(predict)*np.log2(TP/len(predict)) - FP/len(predict)*np.log2(FP/len(predict)) - FN/len(predict)*np.log2(FN/len(predict)) - TN/len(predict)*np.log2(TN/len(predict))

In [45]:
print("Accuracy: ", calcAccuracy(result, testLabels, medLabels))
print("Precision: ", calcPercision(result, testLabels, medLabels))
print("Recall: ", calcRecall(result, testLabels, medLabels))
print("F1 score: ", calcF1(calcPercision(result, testLabels,medLabels), calcRecall(result, testLabels,medLabels)))
print("Conditional entropy: ", calcEntropy(result, testLabels,medLabels))

Accuracy:  0.6495581633049126
Precision:  0.7850061569928829
Recall:  0.6914351894405941
F1 score:  0.735255595738442
Conditional entropy:  1.798102710253255
